Done liao, dont touch

In [4]:
import os
import cv2 as cv
import pickle
import sqlite3
import csv
from insightface.app import FaceAnalysis
from huggingface_hub import HfApi

In [8]:

# Initialize InsightFace with CPU fallback
try:
    app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
except Exception:
    app = FaceAnalysis(providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

def get_file_from_user(folder_path: str) -> str:
    """Validate input folder path."""
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    return folder_path

def process_img(folder_path: str, output_file="database/face_embeddings.pkl"):
    """Extract embeddings from all images in a folder and save to pickle."""
    embeddings_data = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            img = cv.imread(img_path)
            if img is None:
                print(f"[WARNING] Could not read {filename}")
                continue

            img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            faces = app.get(img_rgb)

            if not faces:
                print(f"[WARNING] No face found in {filename}")
                continue

            face = faces[0]  # Take first face
            embedding = face.normed_embedding
            user_id = os.path.splitext(filename)[0]

            embeddings_data.append({
                'id': user_id,
                'image_name': filename,
                'embedding': embedding
            })

    # Save embeddings
    with open(output_file, "wb") as f:
        pickle.dump(embeddings_data, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"[INFO] Saved embeddings to {output_file}")
    return output_file

def save_to_hf(file, repo_id="jesmine0820/assignment_face_recognition", hf_token=None):
    """Upload pickle file to HuggingFace Hub dataset repo."""
    api = HfApi(token=hf_token)
    try:
        api.upload_file(
            path_or_fileobj=file,
            path_in_repo=os.path.basename(file),
            repo_id=repo_id,
            repo_type="dataset"
        )
        print(f"[INFO] Uploaded {file} to {repo_id}")
    except Exception as e:
        print(f"[ERROR] Failed to upload {file} to HuggingFace: {e}")
    
process_img("photos")

c:\Users\User\anaconda3\envs\environment\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

c:\Users\User\anaconda3\envs\environment\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


[INFO] Saved embeddings to database/face_embeddings.pkl


'database/face_embeddings.pkl'

In [10]:
INFO_FILE = "database/information.db"
os.makedirs("database", exist_ok=True)

def create_info_db():
    """Initialize the SQLite database and return connection + cursor."""
    conn = sqlite3.connect(INFO_FILE, check_same_thread=False)
    cursor = conn.cursor()
    cursor.execute('''
            CREATE TABLE IF NOT EXISTS student_info (
                   id TEXT PRIMARY KEY,
                   name TEXT,
                   faculty TEXT,
                   course TEXT,
                   cgpa NUMBER
                   )
                   ''')
    conn.commit()
    return conn, cursor

def insert_from_csv(csv_path):
    conn, cursor = create_info_db()
    with open(csv_path, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            cursor.execute('''
                INSERT OR REPLACE INTO student_info (id, name, faculty, course, cgpa)
                           VALUES(?,?,?,?,?)
                           ''', (
                               row['StudentID'],
                               row['Name'],
                               row['Faculty'],
                               row['Course'],
                               float(row['CGPA'])
                           ))
    conn.commit()
    conn.close()

insert_from_csv("dataset/dataset.csv")